# Home Credit Default Risk

## Feature Engineering

### Introduction

In this project part, we're enhancing the predictive precision of machine learning models for loan default prediction by applying feature engineering techniques and integrating additional data sources into the application_train dataset.

Supplementary datasets like bureau, bureau_balance, previous_application, POS_CASH_balance, credit_card_balance, and installments_payments provide crucial insights into applicants' credit history, behavioral patterns, and repayment behavior. By carefully engineering features and merging these datasets with application_train, we aim to improve the models' understanding of applicant risk profiles, leading to more accurate predictions of loan default probabilities.

#### Importing Needed Libraries

In [1]:
import os
import sys
sys.path.append("../functions")
from functions import get_column_description

import pandas as pd
import numpy as np

### Main Dataset

The application_train dataset contains loan application information identified by SK_ID_CURR. To incorporate additional data from related tables, a new DataFrame is created, new_features, to store engineered features. This DataFrame includes the SK_ID_CURR column from application_train as a reference for merging information from other tables.

In [2]:
app_train = pd.read_csv('../data/original/application_train.csv')
app_test = pd.read_csv('../data/original/application_test.csv')

In [3]:
app_train_IDs = app_train[['SK_ID_CURR']].copy()
app_test_IDs = app_train[['SK_ID_CURR']].copy()
new_features = pd.DataFrame()
new_features['SK_ID_CURR'] = pd.concat([app_train_IDs['SK_ID_CURR'], app_test_IDs['SK_ID_CURR']], ignore_index=True)

### Supplementary Datasets

In [4]:
column_description = pd.read_csv("../data/original/HomeCredit_columns_description.csv", encoding = "latin")

#### Bureau table

This table contains application data from other loans taken by the client from other credit institutions.
The table is linked to application_train and application_test tables via the SK_ID_CURR feature.

In [5]:
bureau = pd.read_csv('../data/original/bureau.csv', encoding = "latin")

In [6]:
print(f"Shape of bureau dataset: {bureau.shape}")

Shape of bureau dataset: (1716428, 17)


In [7]:
if bureau.duplicated().sum() == 0:
    print("No duplicated rows found.")
else:
    print("Duplicated rows found.")

No duplicated rows found.


In [8]:
bureau = bureau.rename(columns={'SK_ID_BUREAU': 'SK_BUREAU_ID'}) #fixing name to match the description table
column_names = bureau.columns.tolist()
print(get_column_description(column_description, column_names))

SK_ID_CURR: ID of loan in our sample
SK_BUREAU_ID: Recoded ID of previous Credit Bureau credit related to our loan (unique coding for each loan application)
CREDIT_ACTIVE: Status of the Credit Bureau (CB) reported credits
CREDIT_CURRENCY: Recoded currency of the Credit Bureau credit
DAYS_CREDIT: How many days before current application did client apply for Credit Bureau credit
CREDIT_DAY_OVERDUE: Number of days past due on CB credit at the time of application for related loan in our sample
DAYS_CREDIT_ENDDATE: Remaining duration of CB credit (in days) at the time of application in Home Credit
DAYS_ENDDATE_FACT: Days since CB credit ended at the time of application in Home Credit (only for closed credit)
AMT_CREDIT_MAX_OVERDUE: Maximal amount overdue on the Credit Bureau credit so far (at application date of loan in our sample)
CNT_CREDIT_PROLONG: How many times was the Credit Bureau credit prolonged
AMT_CREDIT_SUM: Current credit amount for the Credit Bureau credit
AMT_CREDIT_SUM_DEBT:

In [9]:
bureau.head()

,SK_ID_CURR,SK_BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [10]:
sk_id_curr_counts = bureau['SK_ID_CURR'].value_counts()
print(sk_id_curr_counts.head())

SK_ID_CURR
120860    116
169704     94
318065     78
251643     61
425396     60
Name: count, dtype: int64


The dataset reveals that some clients have multiple previous loans and both active and closed credits associated with them. To capture this information, aggregate features such as CNT_PREVIOUS_LOANS, CNT_ACTIVE, and CNT_CLOSED are calculated.

In [11]:
cnt_previous_loans = bureau.groupby('SK_ID_CURR')['SK_BUREAU_ID'].count().rename('CNT_PREVIOUS_LOANS')
cnt_active = bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR')['SK_BUREAU_ID'].count().rename('CNT_ACTIVE')
cnt_closed = bureau[bureau['CREDIT_ACTIVE'] == 'Closed'].groupby('SK_ID_CURR')['SK_BUREAU_ID'].count().rename('CNT_CLOSED')

In [12]:
bureau_new_features = pd.concat([cnt_previous_loans, cnt_active, cnt_closed], axis=1).fillna(0)

In [13]:
avg_values = bureau.groupby('SK_ID_CURR').agg({
    'DAYS_CREDIT': 'mean',
    'CREDIT_DAY_OVERDUE': 'mean',
    'DAYS_CREDIT_ENDDATE': 'mean',
    'DAYS_ENDDATE_FACT': 'mean',
    'AMT_CREDIT_MAX_OVERDUE': 'mean',
    'CNT_CREDIT_PROLONG': 'mean',
    'AMT_CREDIT_SUM': 'mean',
    'AMT_CREDIT_SUM_DEBT': 'mean',
    'AMT_CREDIT_SUM_LIMIT': 'mean',
    'AMT_CREDIT_SUM_OVERDUE': 'mean',
    'AMT_ANNUITY': 'mean'
}).add_prefix('AVG_')
credit_type_counts = pd.get_dummies(bureau['CREDIT_TYPE']).groupby(bureau['SK_ID_CURR']).sum()
credit_type_counts.columns = credit_type_counts.columns.str.upper().str.replace(' ', '_')
credit_currency_dummies = pd.get_dummies(bureau['CREDIT_CURRENCY']).groupby(bureau['SK_ID_CURR']).sum()
credit_currency_dummies.columns = credit_currency_dummies.columns.str.upper().str.replace(' ', '_')
bureau['DEBT_TO_CREDIT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']
bureau_new_features = pd.concat([
    avg_values, 
    credit_type_counts, 
    credit_currency_dummies,  
    bureau.groupby('SK_ID_CURR')['DEBT_TO_CREDIT_RATIO'].mean()
], axis=1)

The engineered features derived from the bureau dataset offer valuable insights into clients' credit behavior. They include counts of previous loans, distinctions between active and closed credits, average credit metrics, distribution of credit types and currencies, and the debt-to-credit ratio. These features provide a comprehensive view of clients' borrowing history, credit utilization patterns, and indebtedness.

#### Bureau_balance table

Contains information about the monthly balances of previous loans that were reported to the Credit Bureau.
The table is designed to to capture the payment history, providing behavioral data of the loaners.
This table is linked to bureau table via the SK_ID_BUREAU feature.

In [14]:
bureau_balance = pd.read_csv('../data/original/bureau_balance.csv', encoding = "latin")

In [15]:
print(f"Shape of bureau_balance dataset: {bureau_balance.shape}")

Shape of bureau_balance dataset: (27299925, 3)


In [16]:
if bureau_balance.duplicated().sum() == 0:
    print("No duplicated rows found.")
else:
    print("Duplicated rows found.")

No duplicated rows found.


In [17]:
bureau_balance = bureau_balance.rename(columns={'SK_ID_BUREAU': 'SK_BUREAU_ID'}) #fixing name to match the description table
column_names = bureau_balance.columns.tolist()
print(get_column_description(column_description, column_names))

SK_BUREAU_ID: Recoded ID of previous Credit Bureau credit related to our loan (unique coding for each loan application)
MONTHS_BALANCE: Month of balance relative to application date (-1 means the freshest balance date)
STATUS: Status of Credit Bureau loan during the month (active, closed, DPD0-30, [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60, 5 means DPD 120+ or sold or written off ] )



In [18]:
bureau_balance.head()

,SK_BUREAU_ID,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [19]:
bureau_merged = pd.merge(bureau_balance, bureau[['SK_BUREAU_ID', 'SK_ID_CURR']], on='SK_BUREAU_ID', how='left')
bureau_merged['SK_ID_CURR'] = bureau_merged['SK_ID_CURR'].astype('Int64')

In [20]:
status_counts = bureau_merged.groupby(['SK_ID_CURR', 'STATUS']).size().unstack(fill_value=0)
status_counts.columns = ['CNT_STATUS_' + col for col in status_counts.columns]
total_unique_counts = bureau_merged.groupby('SK_ID_CURR')['SK_BUREAU_ID'].nunique()

aggregated_data = pd.concat([total_unique_counts, status_counts], axis=1)
aggregated_data.rename(columns={'SK_BUREAU_ID': 'CNT_SK_BUREAU_ID'}, inplace=True)


In [21]:
print(aggregated_data.sample(2).T)

SK_ID_CURR        392250  348989
CNT_SK_BUREAU_ID       1       3
CNT_STATUS_0           9       9
CNT_STATUS_1           0       0
CNT_STATUS_2           0       0
CNT_STATUS_3           0       0
CNT_STATUS_4           0       0
CNT_STATUS_5           0       0
CNT_STATUS_C          33      11
CNT_STATUS_X           1       7


In [22]:
new_features = pd.merge(bureau_new_features, aggregated_data, on='SK_ID_CURR', how='left')

#### Previous_application table

Contains information about previous loans at Home Credit by the same client. It includes details about previous loans and client information at the time of those loans.
This table is linked to application_train and application_test tables via the SK_ID_CURR feature.

In [23]:
previous_application = pd.read_csv('../data/original/previous_application.csv', encoding = "latin")

In [24]:
print(f"Shape of previous_application dataset: {previous_application.shape}")

Shape of previous_application dataset: (1670214, 37)


In [25]:
if previous_application.duplicated().sum() == 0:
    print("No duplicated rows found.")
else:
    print("Duplicated rows found.")

No duplicated rows found.


In [26]:
column_names = previous_application.columns.tolist()
for i in column_names[1:]:
    print(i)
    print(get_column_description(column_description, i))

SK_ID_CURR
SK_ID_CURR: ID of loan in our sample

NAME_CONTRACT_TYPE
NAME_CONTRACT_TYPE: Identification if loan is cash or revolving

AMT_ANNUITY
AMT_ANNUITY: Loan annuity

AMT_APPLICATION
AMT_APPLICATION: For how much credit did client ask on the previous application

AMT_CREDIT
AMT_CREDIT: Credit amount of the loan

AMT_DOWN_PAYMENT
AMT_DOWN_PAYMENT: Down payment on the previous application

AMT_GOODS_PRICE
AMT_GOODS_PRICE: For consumer loans it is the price of the goods for which the loan is given

WEEKDAY_APPR_PROCESS_START
WEEKDAY_APPR_PROCESS_START: On which day of the week did the client apply for the loan

HOUR_APPR_PROCESS_START
HOUR_APPR_PROCESS_START: Approximately at what hour did the client apply for the loan

FLAG_LAST_APPL_PER_CONTRACT
FLAG_LAST_APPL_PER_CONTRACT: Flag if it was last application for the previous contract. Sometimes by mistake of client or our clerk there could be more applications for one single contract

NFLAG_LAST_APPL_IN_DAY
NFLAG_LAST_APPL_IN_DAY: Fla

In [27]:
total_unique_loans = previous_application.groupby('SK_ID_CURR')['SK_ID_PREV'].nunique()

average_values = previous_application.groupby('SK_ID_CURR').agg({
    'AMT_ANNUITY': 'mean',
    'AMT_APPLICATION': 'mean',
    'AMT_DOWN_PAYMENT': 'mean',
    'AMT_GOODS_PRICE': 'mean',
    'CNT_PAYMENT': 'mean',
    'RATE_DOWN_PAYMENT': 'mean',
    'RATE_INTEREST_PRIMARY': 'mean',
    'RATE_INTEREST_PRIVILEGED': 'mean',
    'DAYS_FIRST_DUE': 'mean',
    'DAYS_LAST_DUE_1ST_VERSION': 'mean',
    'DAYS_LAST_DUE': 'mean'
}).add_prefix('AVG_')

category_counts = pd.get_dummies(previous_application[['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'NAME_CONTRACT_STATUS', 'NAME_CLIENT_TYPE']], 
                                 columns=['NAME_CONTRACT_TYPE', 'NAME_CONTRACT_STATUS', 'NAME_CLIENT_TYPE']).groupby('SK_ID_CURR').sum()

category_counts.columns = ['CNT_' + col.replace(' ', '_').upper() for col in category_counts.columns]

avg_cnt_payment = previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].mean().rename('AVG_CNT_PAYMENT')

prev_agg_data = pd.concat([total_unique_loans, average_values, category_counts, avg_cnt_payment], axis=1)

print(prev_agg_data.head(2).T)

SK_ID_CURR                                    100001      100002
SK_ID_PREV                                  1.000000       1.000
AVG_AMT_ANNUITY                          3951.000000    9251.775
AVG_AMT_APPLICATION                     24835.500000  179055.000
AVG_AMT_DOWN_PAYMENT                     2520.000000       0.000
AVG_AMT_GOODS_PRICE                     24835.500000  179055.000
AVG_CNT_PAYMENT                             8.000000      24.000
AVG_RATE_DOWN_PAYMENT                       0.104326       0.000
AVG_RATE_INTEREST_PRIMARY                        NaN         NaN
AVG_RATE_INTEREST_PRIVILEGED                     NaN         NaN
AVG_DAYS_FIRST_DUE                      -1709.000000    -565.000
AVG_DAYS_LAST_DUE_1ST_VERSION           -1499.000000     125.000
AVG_DAYS_LAST_DUE                       -1619.000000     -25.000
CNT_NAME_CONTRACT_TYPE_CASH_LOANS           0.000000       0.000
CNT_NAME_CONTRACT_TYPE_CONSUMER_LOANS       1.000000       1.000
CNT_NAME_CONTRACT_TYPE_RE

#### Installments_payments table

Contains repayment history for previously disbursed credits in Home Credit related to the loans in the sample.
This table includes information about payments made and missed payments.
Each row representts either a payment installment or a missed payment for a previous Home Credit credit.
It is connected to previous_application through the SK_ID_PREV feature. Also it is linked to application_train and application_test tables via the SK_ID_CURR feature.

In [28]:
installments_payments = pd.read_csv('../data/original/installments_payments.csv', encoding = "latin")

In [29]:
print(f"Shape of installments_payments dataset: {installments_payments.shape}")

Shape of installments_payments dataset: (13605401, 8)


In [30]:
if installments_payments.duplicated().sum() == 0:
    print("No duplicated rows found.")
else:
    print("Duplicated rows found.")

No duplicated rows found.


In [31]:
column_names = installments_payments.columns.tolist()
print(get_column_description(column_description, column_names[1:]))

SK_ID_CURR: ID of loan in our sample
NUM_INSTALMENT_VERSION: Version of installment calendar (0 is for credit card) of previous credit. Change of installment version from month to month signifies that some parameter of payment calendar has changed
NUM_INSTALMENT_NUMBER: On which installment we observe payment
DAYS_INSTALMENT: When the installment of previous credit was supposed to be paid (relative to application date of current loan)
DAYS_ENTRY_PAYMENT: When was the installments of previous credit paid actually (relative to application date of current loan)
AMT_INSTALMENT: What was the prescribed installment amount of previous credit on this installment
AMT_PAYMENT: What the client actually paid on previous credit on this installment



In [32]:
installments_payments.sample(3).T

,13191795,13134651,4237727
SK_ID_PREV,2459045.00,2525973.0,2564819.000
SK_ID_CURR,438945.00,410406.0,281271.000
NUM_INSTALMENT_VERSION,1.00,1.0,1.000
NUM_INSTALMENT_NUMBER,22.00,27.0,2.000
DAYS_INSTALMENT,-348.00,-733.0,-2656.000
DAYS_ENTRY_PAYMENT,-383.00,-764.0,-2656.000
AMT_INSTALMENT,45495.81,34341.3,3847.635
AMT_PAYMENT,45495.81,17701.2,3847.635


In [33]:
installments_average_values = installments_payments.groupby('SK_ID_CURR').agg({
    'NUM_INSTALMENT_VERSION': 'mean',
    'NUM_INSTALMENT_NUMBER': 'mean',
    'DAYS_INSTALMENT': 'mean',
    'DAYS_ENTRY_PAYMENT': 'mean',
    'AMT_INSTALMENT': 'mean',
    'AMT_PAYMENT': 'mean'
}).add_prefix('AVG_')

installments_category_counts = pd.get_dummies(installments_payments['NUM_INSTALMENT_VERSION']).groupby(installments_payments['SK_ID_CURR']).sum()
installments_category_counts.columns = ['CNT_NUM_INSTALMENT_VERSION_' + str(col) for col in installments_category_counts.columns]
installments_aggregated_data = pd.concat([installments_average_values, installments_category_counts], axis=1)

In [34]:
prev_agg_data = pd.merge(prev_agg_data, installments_aggregated_data, on='SK_ID_CURR', how='left')

#### POS_CASH_balance table

Contains monthly balance snapshots of previous point-of-sale (POS) and cash loans that the applicant had with Home Credit.
It is connected to previous_application through the SK_ID_PREV feature. Also it is linked to application_train and application_test tables via the SK_ID_CURR feature.

In [35]:
pos_cash_balance = pd.read_csv('../data/original/POS_CASH_balance.csv', encoding = "latin")

In [36]:
print(f"Shape of pos_cash_balance dataset: {pos_cash_balance.shape}")

Shape of pos_cash_balance dataset: (10001358, 8)


In [37]:
if pos_cash_balance.duplicated().sum() == 0:
    print("No duplicated rows found.")
else:
    print("Duplicated rows found.")

No duplicated rows found.


In [38]:
column_names = pos_cash_balance.columns.tolist()
print(get_column_description(column_description, column_names[1:]))

SK_ID_CURR: ID of loan in our sample
MONTHS_BALANCE: Month of balance relative to application date (-1 means the freshest balance date)
CNT_INSTALMENT: Term of previous credit (can change over time)
CNT_INSTALMENT_FUTURE: Installments left to pay on the previous credit
NAME_CONTRACT_STATUS: Contract status during the month
SK_DPD: DPD (days past due) during the month of previous credit
SK_DPD_DEF: DPD during the month with tolerance (debts with low loan amounts are ignored) of the previous credit



In [39]:
pos_cash_balance.sample().T

,3119391
SK_ID_PREV,1670663
SK_ID_CURR,194628
MONTHS_BALANCE,-64
CNT_INSTALMENT,9.0
CNT_INSTALMENT_FUTURE,3.0
NAME_CONTRACT_STATUS,Active
SK_DPD,0
SK_DPD_DEF,0


In [40]:
pos_cash_average_values = pos_cash_balance.groupby('SK_ID_CURR').agg({
    'MONTHS_BALANCE': 'mean',
    'CNT_INSTALMENT': 'mean',
    'CNT_INSTALMENT_FUTURE': 'mean',
    'SK_DPD': 'mean',
    'SK_DPD_DEF': 'mean'
}).add_prefix('AVG_')

contract_status_dummies = pd.get_dummies(pos_cash_balance['NAME_CONTRACT_STATUS']).groupby(pos_cash_balance['SK_ID_CURR']).sum()
contract_status_dummies.columns = ['CNT_NAME_CONTRACT_STATUS_' + col.replace(' ', '_').upper() for col in contract_status_dummies.columns]

pos_cash_aggregated_data = pd.concat([pos_cash_average_values, contract_status_dummies], axis=1)

In [41]:
prev_agg_data = pd.merge(prev_agg_data, pos_cash_aggregated_data, on='SK_ID_CURR', how='left')

#### Credit_card_balance table

Contains monthly balance snapshots of previous credit cards that the applicant had with Home Credit.
This table gives insights into the payment history and balance of previous credit card accounts.
It is connected to previous_application through the SK_ID_PREV feature. Also it is linked to application_train and application_test tables via the SK_ID_CURR feature.

In [42]:
credit_card_balance = pd.read_csv('../data/original/credit_card_balance.csv', encoding = "latin")

In [43]:
print(f"Shape of credit_card_balance dataset: {credit_card_balance.shape}")

Shape of credit_card_balance dataset: (3840312, 23)


In [44]:
if credit_card_balance.duplicated().sum() == 0:
    print("No duplicated rows found.")
else:
    print("Duplicated rows found.")

No duplicated rows found.


In [45]:
column_names = credit_card_balance.columns.tolist()
print(get_column_description(column_description, column_names[1:]))

SK_ID_CURR: ID of loan in our sample
MONTHS_BALANCE: Month of balance relative to application date (-1 means the freshest balance date)
AMT_BALANCE: Balance during the month of previous credit
AMT_CREDIT_LIMIT_ACTUAL: Credit card limit during the month of the previous credit
AMT_DRAWINGS_ATM_CURRENT: Amount drawing at ATM during the month of the previous credit
AMT_DRAWINGS_CURRENT: Amount drawing during the month of the previous credit
AMT_DRAWINGS_OTHER_CURRENT: Amount of other drawings during the month of the previous credit
AMT_DRAWINGS_POS_CURRENT: Amount drawing or buying goods during the month of the previous credit
AMT_INST_MIN_REGULARITY: Minimal installment for this month of the previous credit
AMT_PAYMENT_CURRENT: How much did the client pay during the month on the previous credit
AMT_PAYMENT_TOTAL_CURRENT: How much did the client pay during the month in total on the previous credit
AMT_RECEIVABLE_PRINCIPAL: Amount receivable for principal on the previous credit
AMT_RECIVABL

In [46]:
credit_card_average_values = credit_card_balance.groupby('SK_ID_CURR').agg({
    'MONTHS_BALANCE': 'mean',
    'AMT_BALANCE': 'mean',
    'AMT_CREDIT_LIMIT_ACTUAL': 'mean',
    'AMT_DRAWINGS_ATM_CURRENT': 'mean',
    'AMT_DRAWINGS_CURRENT': 'mean',
    'AMT_DRAWINGS_OTHER_CURRENT': 'mean',
    'AMT_DRAWINGS_POS_CURRENT': 'mean',
    'AMT_INST_MIN_REGULARITY': 'mean',
    'AMT_PAYMENT_CURRENT': 'mean',
    'AMT_PAYMENT_TOTAL_CURRENT': 'mean',
    'AMT_RECEIVABLE_PRINCIPAL': 'mean',
    'AMT_RECIVABLE': 'mean',
    'AMT_TOTAL_RECEIVABLE': 'mean',
    'CNT_DRAWINGS_ATM_CURRENT': 'mean',
    'CNT_DRAWINGS_CURRENT': 'mean',
    'CNT_DRAWINGS_OTHER_CURRENT': 'mean',
    'CNT_DRAWINGS_POS_CURRENT': 'mean',
    'CNT_INSTALMENT_MATURE_CUM': 'mean',
    'SK_DPD': 'mean',
    'SK_DPD_DEF': 'mean'
}).add_prefix('AVG_')

contract_status_dummies = pd.get_dummies(credit_card_balance['NAME_CONTRACT_STATUS']).groupby(credit_card_balance['SK_ID_CURR']).sum()
contract_status_dummies.columns = ['CNT_NAME_CONTRACT_STATUS_CARD' + col.replace(' ', '_').upper() for col in contract_status_dummies.columns]
credit_card_aggregated_data = pd.concat([credit_card_average_values, contract_status_dummies], axis=1)



In [47]:
prev_agg_data = pd.merge(prev_agg_data, credit_card_aggregated_data, on='SK_ID_CURR', how='left')

In [48]:
new_features = pd.merge(new_features, prev_agg_data, on='SK_ID_CURR', how='left')

### Merging the Dataset

In [50]:
app_train_extended = pd.merge(app_train, new_features, on='SK_ID_CURR', how='left')
file_path = '../data/app_train_extended.csv'
app_train_extended.to_csv(file_path, index=False)

app_test_extended = pd.merge(app_test, new_features, on='SK_ID_CURR', how='left')
file_path = '../data/app_test_extended.csv'
app_test_extended.to_csv(file_path, index=False)